In [1]:
import dotenv
import openai
import os
from rich.pretty import pprint

In [2]:
dotenv.load_dotenv()
openaiclient = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
from typing import List, Callable, Dict, Optional
import inspect

def openai_function_wrapper(
    function_description: str,
    parameter_descriptions: Dict[str, str] = {},
) -> Callable:
    def decorator(func: Callable) -> Callable:
        class FunctionWrapper:
            def __init__(self, func, function_description, parameter_descriptions):
                self.func = func
                self.function_description = function_description
                self.parameter_descriptions = parameter_descriptions
                self.output = {
                    "type": "function",
                    "function": {
                        "name": func.__name__,
                        "description": self.function_description,
                        "parameters": {
                            "type": "object",
                            "required": [],
                            "properties": {},
                            "additionalProperties": False,
                        },
                    },
                }
                self._inspect_parameters()

            def __call__(self, *args, **kwargs):
                return self.func(*args, **kwargs)

            def _inspect_parameters(self):
                signature = inspect.signature(self.func)
                for name, param in signature.parameters.items():
                    param_info = {
                        "description": self.parameter_descriptions.get(name, ""),
                        "type": self._get_param_type(param),
                    }
                    self.output["function"]["parameters"]["properties"][
                        name
                    ] = param_info
                    if param.default == inspect.Parameter.empty:
                        self.output["function"]["parameters"]["required"].append(name)

            def _get_param_type(self, param):
                # Basic type mapping
                if param.annotation is inspect.Parameter.empty:
                    return "string"
                elif param.annotation is str:
                    return "string"
                elif param.annotation is int:
                    return "integer"
                elif param.annotation is float:
                    return "number"
                elif param.annotation is bool:
                    return "boolean"
                elif param.annotation is list:
                    return "array"
                else:
                    return "string"

        return FunctionWrapper(func, function_description, parameter_descriptions)

    return decorator

def create_tools_schema(functions: List[callable]) -> List:
    tools = []
    for function in functions:
        tools.append(function.output)
    return tools

def create_tools_list(functions: List[callable]) -> List:
    tools_list = {} 
    for function in functions:
        tools_list[function.funct.__name__] = function
    return tools_list

# function parser
# tool executor

In [5]:
@openai_function_wrapper(
    function_description="Get the weather for a specific location today",
    parameter_descriptions={"location": "The location to get the weather for"},
)
def get_weather_today(location: str) -> str:
    return "The weather is sunny in " + location

@openai_function_wrapper(
    function_description="Get the weather for a specific location in the future",
    parameter_descriptions={
        "location": "The location to get the weather for",
        "days_from_now": "The number of days in the future to get the weather for",
    },
)
def get_weather_forecast(location: str, days_from_now: int) -> str:
    return f"The weather will be sunny in {location} in {days_from_now} days"

In [6]:
tools = create_tools_schema([get_weather_today, get_weather_forecast])
pprint(tools)
tools_list = create_tools_list([get_weather_today, get_weather_forecast])
pprint(tools_list)

[
│   {
│   │   'type': 'function',
│   │   'function': {
│   │   │   'name': 'get_weather_today',
│   │   │   'description': 'Get the weather for a specific location today',
│   │   │   'parameters': {
│   │   │   │   'type': 'object',
│   │   │   │   'required': ['location'],
│   │   │   │   'properties': {
│   │   │   │   │   'location': {'description': 'The location to get the weather for', 'type': 'string'}
│   │   │   │   },
│   │   │   │   'additionalProperties': False
│   │   │   }
│   │   }
│   },
│   {
│   │   'type': 'function',
│   │   'function': {
│   │   │   'name': 'get_weather_forecast',
│   │   │   'description': 'Get the weather for a specific location in the future',
│   │   │   'parameters': {
│   │   │   │   'type': 'object',
│   │   │   │   'required': ['location', 'days_from_now'],
│   │   │   │   'properties': {
│   │   │   │   │   'location': {'description': 'The location to get the weather for', 'type': 'string'},
│   │   │   │   │   'days_from_now': {
│   │   │   │   │   │   'description': 'The number of days in the future to get the weather for',
│   │   │   │   │   │   'type': 'integer'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   'additionalProperties': False
│   │   │   }
│   │   }
│   }
]

{
│   'get_weather_today': <__main__.openai_function_wrapper.<locals>.decorator.<locals>.FunctionWrapper object at 0x000001FF663056A0>,
│   'get_weather_forecast': <__main__.openai_function_wrapper.<locals>.decorator.<locals>.FunctionWrapper object at 0x000001FF66306750>
}

In [8]:
response = openaiclient.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role": "user", "content": "What is the weather in two days?"},
        {"role": "assistant", "content": "I can help you with that. What location are you interested in?"},
    ],
)

pprint(response)

ChatCompletion(
│   id='chatcmpl-AjktQnfje5zfpYHXc1h8G3EH0CQx8',
│   choices=[
│   │   Choice(
│   │   │   finish_reason='stop',
│   │   │   index=0,
│   │   │   logprobs=None,
│   │   │   message=ChatCompletionMessage(
│   │   │   │   content="I can't provide real-time information, including weather forecasts. However, you can easily check the weather for your location in two days by using a reliable weather website or app. Let me know if you need tips on finding a forecast!",
│   │   │   │   refusal=None,
│   │   │   │   role='assistant',
│   │   │   │   audio=None,
│   │   │   │   function_call=None,
│   │   │   │   tool_calls=None
│   │   │   )
│   │   )
│   ],
│   created=1735468380,
│   model='gpt-4o-mini-2024-07-18',
│   object='chat.completion',
│   service_tier=None,
│   system_fingerprint='fp_0aa8d3e20b',
│   usage=CompletionUsage(
│   │   completion_tokens=47,
│   │   prompt_tokens=33,
│   │   total_tokens=80,
│   │   completion_tokens_details=CompletionTokensDetails(
│   │   │   accepted_prediction_tokens=0,
│   │   │   audio_tokens=0,
│   │   │   reasoning_tokens=0,
│   │   │   rejected_prediction_tokens=0
│   │   ),
│   │   prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
│   )
)

In [79]:
pprint(response.choices[0].message)

ChatCompletionMessage(
│   content=None,
│   refusal=None,
│   role='assistant',
│   audio=None,
│   function_call=None,
│   tool_calls=[
│   │   ChatCompletionMessageToolCall(
│   │   │   id='call_Y5ZRxHCt2p7b1oP6nl1Mt8l5',
│   │   │   function=Function(arguments='{"location":"New York","days_from_now":2}', name='get_weather_forecast'),
│   │   │   type='function'
│   │   )
│   ]
)

In [56]:
import json  # To parse JSON strings

def parse_functions(response):
    functions = []
    for tool in response.choices[0].message.tool_calls:
        if tool.type == "function":
            functions.append(tool)
    return functions

def execute_function(function_object, tools_list):
    function = tools_list[function_object.function.name]
    kwargs = json.loads(function_object.function.arguments) 
    result = function(**kwargs)
    return result

def execute_functions(response, tools_list):
    for call in response.choices[0].message.tool_calls:
        function = tools_list[call.function.name]
        kwargs = json.loads(call.function.arguments) 
        result = function(**kwargs) 

execute_functions(response, tools_list)



In [70]:
parse_functions(response)

[ChatCompletionMessageToolCall(id='call_Y5ZRxHCt2p7b1oP6nl1Mt8l5', function=Function(arguments='{"location":"New York","days_from_now":2}', name='get_weather_forecast'), type='function')]

In [60]:
result = execute_function(parse_functions(response)[0], tools_list)

In [78]:
pprint(response.choices[0].message)

ChatCompletionMessage(
│   content=None,
│   refusal=None,
│   role='assistant',
│   audio=None,
│   function_call=None,
│   tool_calls=[
│   │   ChatCompletionMessageToolCall(
│   │   │   id='call_Y5ZRxHCt2p7b1oP6nl1Mt8l5',
│   │   │   function=Function(arguments='{"location":"New York","days_from_now":2}', name='get_weather_forecast'),
│   │   │   type='function'
│   │   )
│   ]
)

In [77]:
response2 = openaiclient.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role": "user", "content": "What is the weather in two days?"},
        {"role": "assistant", "content": "I can help you with that. What location are you interested in?"},
        {"role": "user", "content": "New York"},
        response.choices[0].message,
        {"role": "tool",
         "tool_call_id": parse_functions(response)[0].id,
         "name": parse_functions(response)[0].function.name,
         "content": result}],
    tools = tools
)

pprint(response2)

ChatCompletion(
│   id='chatcmpl-AiGfwYDrwi1Lu5iGCsk5ZzgKgvfQf',
│   choices=[
│   │   Choice(
│   │   │   finish_reason='stop',
│   │   │   index=0,
│   │   │   logprobs=None,
│   │   │   message=ChatCompletionMessage(
│   │   │   │   content='In two days, the weather in New York is expected to be sunny.',
│   │   │   │   refusal=None,
│   │   │   │   role='assistant',
│   │   │   │   audio=None,
│   │   │   │   function_call=None,
│   │   │   │   tool_calls=None
│   │   │   )
│   │   )
│   ],
│   created=1735113896,
│   model='gpt-4o-mini-2024-07-18',
│   object='chat.completion',
│   service_tier=None,
│   system_fingerprint='fp_0aa8d3e20b',
│   usage=CompletionUsage(
│   │   completion_tokens=17,
│   │   prompt_tokens=188,
│   │   total_tokens=205,
│   │   completion_tokens_details=CompletionTokensDetails(
│   │   │   accepted_prediction_tokens=0,
│   │   │   audio_tokens=0,
│   │   │   reasoning_tokens=0,
│   │   │   rejected_prediction_tokens=0
│   │   ),
│   │   prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
│   )
)

In [ ]:
# loader 

class runtime():
    def __init__(self, instructions: List[str]):
        self.instructions = instructions 
        self.tasks = []
    
    def run(self):
        while self.instructions:

            task = self.instructions.pop(0)
            self.tasks.append(task)
            response = openaiclient.chat.completions.create(
                model = "gpt-4o-mini",
                messages = [
                    {"role": "user", "content": task},
                ],
                tools = tools

def runtime(instructions: List):
    instructions = instructions
    tasks = [] 
    while True:
        if instructions:
            openaiclient.chat.completions.create(
                model = "gpt-4o-mini",
                messages = [
                    {"role": "user", "content": instructions[0]},
                ],
                tools = tools
            )
            task = instructions.pop(0)

            tasks.append(task)
            response = openaiclient.chat.completions.create(
                model = "gpt-4o-mini",
                messages = [
                    {"role": "user", "content": task},
                ],
                tools = tools



In [ ]:
from functools import wraps



def openai_function(function):
    @wraps(function)
    def wrapper(*args, **kwargs):
        return function(*args, **kwargs)

In [6]:
openai_response = openaiclient.chat.completions.create(model="gpt-4o-mini", 
                                           messages=[
                                               {"role": "user", 
                                                "content": "What is the capital of France?"}
                                           ])

In [34]:
pprint(openai_response)

ChatCompletion(
│   id='chatcmpl-AcOXmxgLNdMGTu1sxnBMisN4gKCw3',
│   choices=[
│   │   Choice(
│   │   │   finish_reason='stop',
│   │   │   index=0,
│   │   │   logprobs=None,
│   │   │   message=ChatCompletionMessage(
│   │   │   │   content='The capital of France is Paris.',
│   │   │   │   refusal=None,
│   │   │   │   role='assistant',
│   │   │   │   audio=None,
│   │   │   │   function_call=None,
│   │   │   │   tool_calls=None
│   │   │   )
│   │   )
│   ],
│   created=1733714174,
│   model='gpt-4o-mini-2024-07-18',
│   object='chat.completion',
│   service_tier=None,
│   system_fingerprint='fp_bba3c8e70b',
│   usage=CompletionUsage(
│   │   completion_tokens=7,
│   │   prompt_tokens=14,
│   │   total_tokens=21,
│   │   completion_tokens_details=CompletionTokensDetails(
│   │   │   accepted_prediction_tokens=0,
│   │   │   audio_tokens=0,
│   │   │   reasoning_tokens=0,
│   │   │   rejected_prediction_tokens=0
│   │   ),
│   │   prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
│   )
)

In [35]:
pprint(anthropic_response)

Message(
│   id='msg_01Rk6CdgwMAVKnrvKuqifzak',
│   content=[TextBlock(text='The capital of France is Paris.', type='text')],
│   model='claude-3-5-haiku-20241022',
│   role='assistant',
│   stop_reason='end_turn',
│   stop_sequence=None,
│   type='message',
│   usage=Usage(input_tokens=14, output_tokens=10)
)

In [31]:
from typing import Any, List
from dataclasses import dataclass
from openai.types.chat import ChatCompletionMessage
from anthropic.types import TextBlock, ToolUseBlock

@dataclass
class Choice:
    finish_reason: str
    index: int
    logprobs: Any
    message: Any

@dataclass
class Usage:
    input_tokens: int
    output_tokens: int

@dataclass
class ResponseObject:
    id: str
    choices: List[Choice]
    model: str
    usage: Usage

def parse_openai(response):
    return ResponseObject(
        id=response.id,
        choices=[
            Choice(
                finish_reason=choice.finish_reason,
                index=choice.index,
                logprobs=choice.logprobs,
                message=choice.message
            ) for choice in response.choices
        ],
        model=response.model,
        usage=Usage(
            input_tokens=response.usage.prompt_tokens,
            output_tokens=response.usage.completion_tokens,
        )
    )

def parse_anthropic(response):
    message = None
    tool = None
    for block in response.content:
        if isinstance(block, TextBlock):
            message = block.text
        elif isinstance(block, ToolUseBlock):
            tool = block
    return ResponseObject(
        id=response.id,
        model=response.model,
        usage=Usage(
            input_tokens=response.usage.input_tokens,
            output_tokens=response.usage.output_tokens,
        ),
        choices=[Choice(
            finish_reason=response.stop_reason,
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=message,
                refusal=response.stop_reason,
                role=response.role,
                audio=None,
                function_call=None,
                tool_calls=tool
            )
        )]
    )

pprint(parse_anthropic(anthropic_response))

ResponseObject(
│   id='msg_01YPgxa4UeV1MxP8c5KmkyhU',
│   choices=[
│   │   Choice(
│   │   │   finish_reason='end_turn',
│   │   │   index=0,
│   │   │   logprobs=None,
│   │   │   message=ChatCompletionMessage(
│   │   │   │   content='The capital of France is Paris.',
│   │   │   │   refusal='end_turn',
│   │   │   │   role='assistant',
│   │   │   │   audio=None,
│   │   │   │   function_call=None,
│   │   │   │   tool_calls=None
│   │   │   )
│   │   )
│   ],
│   model='claude-3-5-haiku-20241022',
│   usage=Usage(input_tokens=14, output_tokens=10)
)

In [34]:
from openai import OpenAI
from typing import List, Dict, Any
import json
from datetime import datetime

# Initialize OpenAI client
client = OpenAI()

# Define a function to get the current weather (mock implementation)
def get_weather(location: str, unit: str = "celsius") -> Dict[str, Any]:
    # This would normally call a weather API
    return {
        "location": location,
        "temperature": 22,
        "unit": unit,
        "condition": "sunny"
    }

# Define a function to get the current time
def get_current_time(timezone: str = "UTC") -> str:
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Define available functions
functions = [
    {
        "name": "get_weather",
        "description": "Get the current weather for a location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g., San Francisco, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit"
                }
            },
            "required": ["location"]
        }
    }
]

# Define available tools (newer approach)
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current time, optionally in a specific timezone",
            "parameters": {
                "type": "object",
                "properties": {
                    "timezone": {
                        "type": "string",
                        "description": "The timezone (e.g., UTC, EST)"
                    }
                }
            }
        }
    }
]

def demonstrate_function_calling():
    print("\n=== Function Calling Demo ===")
    
    response = parse_openai(client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": "What's the weather like in London?"}
        ],
        functions=functions,
        function_call="auto"
    ))

    pprint(response)
    # Get the response message
    message = response.choices[0].message

    # Check if the model wants to call a function
    if message.function_call:
        # Get the function name and arguments
        func_name = message.function_call.name
        func_args = json.loads(message.function_call.arguments)
        
        # Call the function
        if func_name == "get_weather":
            function_response = get_weather(**func_args)
            
            print(message)
            pprint([
                    {"role": "user", "content": "What's the weather like in London?"},
                    message.content,
                    {
                        "role": "function",
                        "name": "get_weather",
                        "content": json.dumps(function_response)
                    }
                ])
            # Get the final response from the model
            final_response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "user", "content": "What's the weather like in London?"},
                    message,
                    {
                        "role": "function",
                        "name": "get_weather",
                        "content": json.dumps(function_response)
                    }
                ]
            )
            
            pprint(final_response)

            print("Final Response:", final_response.choices[0].message.content)

demonstrate_function_calling()


=== Function Calling Demo ===


ResponseObject(
│   id='chatcmpl-AcUwilv7zxjuzOIoesoS7X5I3cqdt',
│   choices=[
│   │   Choice(
│   │   │   finish_reason='function_call',
│   │   │   index=0,
│   │   │   logprobs=None,
│   │   │   message=ChatCompletionMessage(
│   │   │   │   content=None,
│   │   │   │   refusal=None,
│   │   │   │   role='assistant',
│   │   │   │   audio=None,
│   │   │   │   function_call=FunctionCall(arguments='{"location":"London, UK"}', name='get_weather'),
│   │   │   │   tool_calls=None
│   │   │   )
│   │   )
│   ],
│   model='gpt-4o-2024-08-06',
│   usage=Usage(input_tokens=81, output_tokens=16)
)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"location":"London, UK"}', name='get_weather'), tool_calls=None)


[
│   {'role': 'user', 'content': "What's the weather like in London?"},
│   None,
│   {
│   │   'role': 'function',
│   │   'name': 'get_weather',
│   │   'content': '{"location": "London, UK", "temperature": 22, "unit": "celsius", "condition": "sunny"}'
│   }
]

ChatCompletion(
│   id='chatcmpl-AcUwjR5NVgWFt83h8O2vbI6NoETBe',
│   choices=[
│   │   Choice(
│   │   │   finish_reason='stop',
│   │   │   index=0,
│   │   │   logprobs=None,
│   │   │   message=ChatCompletionMessage(
│   │   │   │   content='The weather in London, UK, is currently sunny with a temperature of 22°C.',
│   │   │   │   refusal=None,
│   │   │   │   role='assistant',
│   │   │   │   audio=None,
│   │   │   │   function_call=None,
│   │   │   │   tool_calls=None
│   │   │   )
│   │   )
│   ],
│   created=1733738785,
│   model='gpt-4o-2024-08-06',
│   object='chat.completion',
│   service_tier=None,
│   system_fingerprint='fp_90354628f2',
│   usage=CompletionUsage(
│   │   completion_tokens=18,
│   │   prompt_tokens=66,
│   │   total_tokens=84,
│   │   completion_tokens_details=CompletionTokensDetails(
│   │   │   accepted_prediction_tokens=0,
│   │   │   audio_tokens=0,
│   │   │   reasoning_tokens=0,
│   │   │   rejected_prediction_tokens=0
│   │   ),
│   │   prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
│   )
)

Final Response: The weather in London, UK, is currently sunny with a temperature of 22°C.


In [17]:
anthropicclient.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, world"}
    ]
)

Message(id='msg_01BQMRfcVdEhgeDtrQMG1DGb', content=[TextBlock(text='Hi! How can I help you today?', type='text')], model='claude-3-5-sonnet-20241022', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=10, output_tokens=12))